In [1]:
# for DL modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# for number-crunching
import numpy as np
import scipy.stats as stats

# for dataset management
import os
import pandas as pd

# for timing computations
import time

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
display.set_matplotlib_formats('svg')

# for DL processing
import torch.optim as optim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# for printing out status reports
import sys

# for warnings
import warnings
warnings.filterwarnings("ignore")

C:\Users\athorat\AppData\Local\anaconda3\envs\myGPUEnv\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
C:\Users\athorat\AppData\Local\anaconda3\envs\myGPUEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\athorat\AppData\Local\anaconda3\envs\myGPUEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\athorat\AppData\Local\anaconda3\envs\myGPUEnv\lib\site-packages\tensorflow\python\fra

In [2]:
os.chdir('C:/Users/athorat/OneDrive - Nice Systems Ltd/00_Amit Thorat Data/ISB - AMPBA/01_Project/Term4_DL\Assignment\Part 2')
spamData = pd.read_csv('SPAM text message 20170820 - Data.csv')
spamData.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Calculate the maximum number of words in the 'Message' column
max_words = spamData['Message'].str.split().apply(len).max()
print("Maximum number of words in 'Message' column:", max_words)

Maximum number of words in 'Message' column: 171


In [4]:
# Calculate the maximum length of a sentence in the 'Message' column
max_length = spamData['Message'].str.len().max()
print("Maximum length of a sentence in 'Message' column:", max_length)

Maximum length of a sentence in 'Message' column: 910


In [5]:
# Split the data into training and testing sets
X = spamData['Message']
y = spamData['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Tokenize the text data
max_words = 300
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [7]:
# Convert text data to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [8]:
# Pad sequences to a fixed length (you can choose a suitable max_length)
max_length = 1000  # You can adjust this based on your dataset and computing resources
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [9]:
# Convert 'spam' to 1 and 'ham' to 0 in your target labels
y_train = y_train.apply(lambda x: 1 if x == 'spam' else 0)
y_test = y_test.apply(lambda x: 1 if x == 'spam' else 0)

train_data = TensorDataset(torch.from_numpy(X_train_padded).long(), torch.from_numpy(y_train.values).long())
test_data = TensorDataset(torch.from_numpy(X_test_padded).long(), torch.from_numpy(y_test.values).long())


# Create a PyTorch DataLoader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0], shuffle=True)

In [10]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
device

device(type='cpu')

In [12]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out

In [13]:
# Initialize the model
input_size = max_words  # Size of your vocabulary
hidden_size = 128
output_size = 2  # Two classes: spam or ham
num_layers = 3
model = RNNModel(input_size, hidden_size, output_size, num_layers)

In [14]:
# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# Training loop
num_epochs = 10
batch_size = 64
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / (batch_idx + 1)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}")

Epoch [1/10] - Loss: 0.4005
Epoch [2/10] - Loss: 0.3977
Epoch [3/10] - Loss: 0.3966
Epoch [4/10] - Loss: 0.3954
Epoch [5/10] - Loss: 0.3960
Epoch [6/10] - Loss: 0.3968


KeyboardInterrupt: 

In [ ]:
# Evaluation on the testing set
model.eval()  # Set the model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy on the testing set: {accuracy * 100:.2f}%")

In [ ]:
# Define hyperparameters and architecture options for experimentation
param_grid = {
    'hidden_size': [128, 256],
    'num_layers': [2, 4],
    'learning_rate': [0.01, 0.1],
    'num_epochs': [5, 15]
}

In [ ]:
# Initialize the RNN model
input_size = max_words  # Size of your vocabulary
output_size = 2  # Two classes: spam or ham

# Create a PyTorch DataLoader for the training data
batch_size = 128
train_data = TensorDataset(torch.from_numpy(X_train_padded).long(), torch.from_numpy(y_train.values).long())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
# Perform grid search for hyperparameters and architecture
best_accuracy = 0.0
best_params = {}

for hidden_size in param_grid['hidden_size']:
    for num_layers in param_grid['num_layers']:
        for learning_rate in param_grid['learning_rate']:
            for num_epochs in param_grid['num_epochs']:
                print(f"Testing hyperparameters: hidden_size={hidden_size}, num_layers={num_layers}, "
                      f"learning_rate={learning_rate}, num_epochs={num_epochs}")
                
                # Create and train a model with current hyperparameters
                model = RNNModel(input_size, hidden_size, output_size, num_layers)
                current_accuracy = train_and_evaluate_model(model, train_loader, test_loader, num_epochs, learning_rate)
                
                # Check if current model is the best
                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    best_params = {
                        'hidden_size': hidden_size,
                        'num_layers': num_layers,
                        'learning_rate': learning_rate,
                        'num_epochs': num_epochs
                    }

print(f"Best accuracy: {best_accuracy * 100:.2f}%")
print("Best hyperparameters:", best_params)

In [122]:
# Convert input data X to the correct data type (e.g., torch.float32)
X = X.to(torch.float32)

# Define your RNN model class
class RNNnet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        # Linear layer for output
        self.fc = nn.Linear(hidden_size, 2)  # Output has 2 classes (e.g., spam or ham)

    def forward(self, x):
        # Initialize hidden state with zeros
        #h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward pass through RNN layer
        #out, _ = self.rnn(x, h0)
        out, _ = self.rnn(x)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [123]:
# Initialize the model
input_size = max_length  # Size of your vocabulary
hidden_size = 128
num_layers = 3
net = RNNnet(input_size, hidden_size, num_layers)

In [124]:
# Define loss and optimizer
lossfun = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [125]:
# Training loop
numepochs = 30

for epoch in range(numepochs):
    net.train()
    total_loss = 0

    for X, y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = net(X)

        # Calculate loss
        loss = lossfun(outputs, y)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(train_loader)

    # Print the epoch and average loss
    print(f"Epoch [{epoch + 1}/{numepochs}] - Loss: {avg_loss:.4f}")

RuntimeError: mat1 and mat2 must have the same dtype

In [94]:
class RNNnet(nn.Module):
  def __init__(self,input_size,num_hidden,num_layers):
    super().__init__()

    # store parameters
    self.input_size = input_size
    self.num_hidden = num_hidden
    self.num_layers = num_layers

    # RNN Layer
    self.rnn = nn.RNN(input_size,num_hidden,num_layers)
    
    # linear layer for output
    self.out = nn.Linear(num_hidden,2)
  
  def forward(self,X,num_hidden):
    
    # initialize hidden state for first input
    hidden = torch.zeros(self.num_layers,batch_size,self.num_hidden)

    # run through the RNN layer
    y,hidden = self.rnn(X,num_hidden)
    
    # pass the RNN output through the linear output layer
    o = self.out(y)

    return o,hidden

In [95]:
# Create an instance of the model and inspect
net = RNNnet(input_size, hidden_size, num_layers)

# and check out all learnable parameters
for p in net.named_parameters():
    print(f'{p[0]} has size {list(p[1].shape)}')

rnn.weight_ih_l0 has size [128, 1000]
rnn.weight_hh_l0 has size [128, 128]
rnn.bias_ih_l0 has size [128]
rnn.bias_hh_l0 has size [128]
rnn.weight_ih_l1 has size [128, 128]
rnn.weight_hh_l1 has size [128, 128]
rnn.bias_ih_l1 has size [128]
rnn.bias_hh_l1 has size [128]
rnn.weight_ih_l2 has size [128, 128]
rnn.weight_hh_l2 has size [128, 128]
rnn.bias_ih_l2 has size [128]
rnn.bias_hh_l2 has size [128]
out.weight has size [2, 128]
out.bias has size [2]


In [96]:
# Initialize the model
input_size = max_length  # Size of your vocabulary
hidden_size = 128
#output_size = 2  # Two classes: spam or ham
num_layers = 3
net = RNNnet(input_size, hidden_size, num_layers)

In [97]:
# Define loss and optimizer
lossfun = nn.NLLLoss()  # Negative Log Likelihood Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [87]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}")

KeyboardInterrupt: 

In [104]:
# number of training epochs
numepochs = 30

# initialize losses
losses = np.zeros(numepochs)
signaccuracy = np.zeros(numepochs)

# loop over epochs
for epochi in range(numepochs):
    
    # loop over data segments
    seglosses = []
    segacc    = []
    hiddenstate = None
    
    net.train()
    for X, y in train_loader:
        
        # Ensure X and hidden have the same data type
        #X = X.to(hiddenstate.dtype)
        if hiddenstate is None:
            hiddenstate = torch.zeros(net.num_layers, X.size(0), net.num_hidden).to(X.device)  # Initialize hiddenstate
        
        # Ensure X and hidden have compatible dimensions
        X = X.transpose(0, 1)  # Swap batch and sequence dimensions
        
        # forward pass and loss
        yHat, hiddenstate = net(X, hiddenstate)
        finalValue = yHat[-1]
        loss = lossfun(finalValue,y)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        seglosses.append(loss.item() )
        
    # average losses from this epoch
    losses[epochi] = np.mean(seglosses)
    msg = f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}"
    sys.stdout.write('\r' + msg)
    

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [ ]:
## let's see how the model did!

plt.plot(losses,'s-')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model loss')

plt.show()

In [16]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        
        ## input layer
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Use only the last output in the sequence
        out = self.softmax(out)
        return out

In [17]:
# Initialize the model
input_size = max_words  # Size of your vocabulary
hidden_size = 128
output_size = 2  # Two classes: spam or ham
num_layers = 3
model = RNNModel(input_size, hidden_size, output_size, num_layers)

In [19]:
# Define loss and optimizer
criterion = nn.NLLLoss()  # Negative Log Likelihood Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}")

Epoch [1/10] - Loss: 0.4019
Epoch [2/10] - Loss: 0.3965
Epoch [3/10] - Loss: 0.3990


KeyboardInterrupt: 

In [ ]:
# Evaluation on the testing set
model.eval()  # Set the model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy on the testing set: {accuracy * 100:.2f}%")

In [ ]:
# Define a function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, num_epochs, learning_rate):
    # Define loss and optimizer
    criterion = nn.NLLLoss()  # Negative Log Likelihood Loss for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Calculate and print the average loss for this epoch
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {avg_loss:.4f}")

    # Evaluation on the testing set
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

In [ ]:
# Define hyperparameters and architecture options for experimentation
param_grid = {
    'hidden_size': [64, 128, 256],
    'num_layers': [1, 2, 3],
    'learning_rate': [0.001, 0.01, 0.1],
    'num_epochs': [5, 10, 15, 20]
}

In [ ]:
# Initialize the RNN model
input_size = max_words  # Size of your vocabulary
output_size = 2  # Two classes: spam or ham

# Create a PyTorch DataLoader for the training data
batch_size = 64
train_data = TensorDataset(torch.from_numpy(X_train_padded).long(), torch.from_numpy(y_train.values).long())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
# Perform grid search for hyperparameters and architecture
best_accuracy = 0.0
best_params = {}

for hidden_size in param_grid['hidden_size']:
    for num_layers in param_grid['num_layers']:
        for learning_rate in param_grid['learning_rate']:
            for num_epochs in param_grid['num_epochs']:
                print(f"Testing hyperparameters: hidden_size={hidden_size}, num_layers={num_layers}, "
                      f"learning_rate={learning_rate}, num_epochs={num_epochs}")
                
                # Create and train a model with current hyperparameters
                model = RNNModel(input_size, hidden_size, output_size, num_layers)
                current_accuracy = train_and_evaluate_model(model, train_loader, test_loader, num_epochs, learning_rate)
                
                # Check if current model is the best
                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    best_params = {
                        'hidden_size': hidden_size,
                        'num_layers': num_layers,
                        'learning_rate': learning_rate,
                        'num_epochs': num_epochs
                    }

print(f"Best accuracy: {best_accuracy * 100:.2f}%")
print("Best hyperparameters:", best_params)